In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# scintific computing libraries
import pandas as pd                                      
import numpy as np                    
from scipy import optimize, stats        

# visualisation libraries
import matplotlib.pyplot as plt                      
import seaborn as sns                

# algorithmic library
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn import svm                 
from sklearn.linear_model import LogisticRegression                              
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, RandomizedSearchCV, GridSearchCV  
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, log_loss, mean_squared_error

%matplotlib inline

## Data Acquisition

In [ ]:
df_train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df_test =  pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
df_submission = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')

In [ ]:
print(df_train.shape)
df_train.head()

In [ ]:
print(df_test.shape)
df_test.head()

In [ ]:
print(df_submission.shape)
df_submission.head()

## Data Wrangling

In [ ]:
df_train.describe()

In [ ]:
def data_analysis(df1,  df2):
    print(df1.shape, df2.shape)
    train_dtype = []
    train_isnull = []
    train_unique = []
    test_dtype = []
    test_isnull = []
    test_unique = []
    for col in df2.columns:
        train_dtype.append(df1[col].dtypes)
        train_isnull.append(df1[col].isnull().sum())
        train_unique.append(df1[col].unique().shape[0])
        test_dtype.append(df2[col].dtypes)
        test_isnull.append(df2[col].isnull().sum())
        test_unique.append(df2[col].unique().shape[0])

    df = pd.DataFrame({'train_dtype':train_dtype,'test_dtype':test_dtype,'train_isnull':train_isnull,'test_isnull':test_isnull,
                       'train_unique':train_unique,'test_unique':test_unique}, index=df2.columns)
    df.sort_values(['train_isnull'], axis=0, ascending=False, inplace=True)
    return df

In [ ]:
data_analysis(df_train, df_test).head(50)

In [ ]:
col_drop = []
for col in df_test.columns:
    if (df_train[col].isnull().sum() > 45) or (df_test[col].isnull().sum() > 45):
        col_drop.append(col)
df_train.drop(col_drop, axis=1, inplace=True)
df_test.drop(col_drop, axis=1, inplace=True)

In [ ]:
for col in df_test.columns:
    if df_train[col].unique().shape[0] <= 30:
        df_train[col].fillna(df_train[col].mode()[0], inplace=True)
        df_test[col].fillna(df_test[col].mode()[0], inplace=True)
        
        df_train[col]= LabelEncoder().fit_transform(df_train[col]) 
        df_test[col]= LabelEncoder().fit_transform(df_test[col]) 
        #print(col, df_train[col].unique().shape[0])

In [ ]:
for col in df_test.columns:
    if (df_train[col].isnull().sum() > 0) or (df_test[col].isnull().sum() > 0):
        df_train[col].fillna(df_train[col].median(), inplace=True)
        df_test[col].fillna(df_test[col].median(), inplace=True)

In [ ]:
df_train.head()

In [ ]:
df_test.head()

## Data Exploration

In [ ]:
df_train.corr()

In [ ]:
for col in df_train.columns:
    pearson_coef, p_value = stats.pearsonr(df_train[col], df_train['SalePrice'])
    print(col, pearson_coef, p_value)

In [ ]:
col_continues = []
for col in df_test.columns:
    if df_train[col].unique().shape[0] >= (df_train.shape[0])//10:
        col_continues.append(col)
print(col_continues)

In [ ]:
col_skew = df_train[col_continues].apply(lambda x: stats.skew(x)).sort_values(ascending=False)
high_skew = col_skew[abs(col_skew) > 0.5]
high_skew

In [ ]:
'''for col in high_skew.index:
    df_train[col] = np.log1p(df_train[col])
    df_test[col] = np.log1p(df_test[col])'''

In [ ]:
sns.distplot(df_train['LotArea'])

In [ ]:
q = df_train['LotArea'].quantile(0.99)
df_train = df_train[df_train['LotArea'] < q]
df_train.reset_index(drop=True, inplace=True)

In [ ]:
sns.distplot(df_train['BsmtFinSF1'])

In [ ]:
sns.distplot(df_train['BsmtUnfSF'])

In [ ]:
sns.distplot(df_train['TotalBsmtSF'])

In [ ]:
sns.distplot(df_train['1stFlrSF'])

In [ ]:
sns.distplot(df_train['GrLivArea'])

## Model Development

In [ ]:
Y_train = df_train['SalePrice'].values
X_train = df_train.drop(['Id','SalePrice'], axis = 1)
X_test = df_test.drop(['Id'], axis = 1)

print(X_train.shape, Y_train.shape)
print(X_test.shape)

In [ ]:
'''X_train = StandardScaler().fit(X_train).transform(X_train)
X_test = StandardScaler().fit(X_test).transform(X_test)'''

In [ ]:
x_train, x_test, y_train, y_test = train_test_split( X_train, Y_train, test_size=0.2, random_state=4)
print ('Train set:', x_train.shape,  y_train.shape)
print ('Test set:', x_test.shape,  y_test.shape)

In [ ]:
LR = LinearRegression()
LR.fit(x_train,y_train)

print('Train set mse:', mean_squared_error(y_train, LR.predict(x_train)))
print('Test set mse:', mean_squared_error(y_test, LR.predict(x_test)))
print('Test set score:', LR.score(x_test,y_test))

In [ ]:
RFR = RandomForestRegressor(n_estimators = 1000)
RFR.fit(x_train,y_train)

print('Train set mse:', mean_squared_error(y_train, RFR.predict(x_train)))
print('Test set mse:', mean_squared_error(y_test, RFR.predict(x_test)))
print('Test set score:', RFR.score(x_test,y_test))

In [ ]:
GBR = GradientBoostingRegressor(learning_rate=0.05, max_depth=4, n_estimators=1000)
GBR.fit(x_train,y_train)

print('Train set mse:', mean_squared_error(y_train, GBR.predict(x_train)))
print('Test set mse:', mean_squared_error(y_test, GBR.predict(x_test)))
print('Test set score:', GBR.score(x_test,y_test))

In [ ]:
'''model = GradientBoostingRegressor()
params={ 'learning_rate'    : [0.05, 0.10, 0.15],
         'max_depth'        : [3, 4, 5, 6, 8, 10, 12, 15],
         'n_estimators'     : [100, 500, 1000]}

random_search = RandomizedSearchCV(model, params, cv=5)
random_search.fit(x_train, y_train)
print(random_search.best_estimator_)
print(random_search.best_params_)
print(random_search.best_score_)'''

In [ ]:
prediction = GBR.predict(X_test)

## Submission

In [ ]:
df_submission['SalePrice'] = prediction
df_submission.to_csv('prediction.csv', index=False)
df_submission.head()